In [1]:
import cv2
import numpy as np
min_confidence = 0.5

In [2]:
# 모델 로딩
net = cv2.dnn.readNet("yolo/yolov3.weights", "yolo/yolov3.cfg")

In [3]:
# 화면에 표시될 사물의 이름들을 기록한 파일을 열어서 로딩, 저장해둡니다
with open("yolo/coco.names", "r")as f:
    classes = [line.strip() for line in f.readlines()]
    print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [4]:
# 랜덤의 색상 결정
colors = np.random.uniform(0, 255, size=(len(classes), 3))
print(colors)

[[138.76252197  94.51908763  32.35830485]
 [ 91.63851452 103.68945857   4.88476922]
 [110.34011773 148.80243748  53.51295001]
 [100.03143954  94.14607557  65.22162556]
 [ 45.42500275 246.70618628 126.50625312]
 [102.43132475  50.25902681 141.51531389]
 [110.48974537  79.44393509  59.53183826]
 [242.62994198 241.63424363 205.52524958]
 [130.38881237 193.43506387 113.48700899]
 [216.21124234  68.343151   176.88522907]
 [134.56314309 217.07896553 233.92968916]
 [ 72.25614487 116.74760559 100.81610047]
 [ 41.89495373  18.50489921  61.31135642]
 [100.76257752 145.31445912  76.48138883]
 [ 83.98992601  54.91798571 114.81488064]
 [ 71.46142719 223.6381611   40.48453118]
 [ 49.68610991 124.98921561 181.30473247]
 [139.61183312 151.05449913 240.01634833]
 [ 82.45153555   8.74445565 158.28181661]
 [ 73.13339177 144.05566249  49.52774721]
 [ 65.33130661 148.5152322   27.85216615]
 [ 97.13557228  11.23101922  21.97701114]
 [ 24.39216915 140.69606984  16.0273057 ]
 [233.73328298 209.23399815 251.94

In [5]:
# net 변수에 로딩된 모델에서, 출력양식(인식된 사물의 추가정보) 설정 내용을 추출합니다
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [6]:
# 이미지 로딩 & 크기 추출
img = cv2.imread("image/yolo_01.jpg")
height, width, channels = img.shape

In [7]:
# 이미지 형식 변환
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0,0,0), True, crop=False)
# 모델에 이미지 set
net.setInput(blob)
# 이미지 내부에 있는 사물 인식 실행
outs = net.forward(output_layers)
# 표시할 사물의 사각형 및 확률 등을 저장할 리스트 생성
class_ids = []
confidences = []
boxes = []

In [8]:
print(outs[0].shape)

(507, 85)


In [9]:
# 인식된 사물의 개수만큼 반복실행
for out in outs:
    # 인식된 사물의 판별 정보별로 반복실행
    for detection in out:
        # 인식 가능한 사물일 가능성이 담긴 확률들을 먼저 추출
        scores = detection[5:]
        # 확률들을 정렬하고 가장 높은 확률의 인덱스 추출
        class_id = np.argmax(scores)
        # 해당 인덱스의 확률 추출
        confidence = scores[class_id]
        # 추출한 확률이 50%(0.5 이상인 경우)
        if confidence > min_confidence:
            # 해당 사물의 중점 좌표 계산
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            # 가로 세로 길이 및 시작 좌표 계산
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            # 계산된 좌표 정보 및 확률 그리고 사물이름(class_id) 정보를 각 리스트에 저장합니다
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [10]:
class_ids

[2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 3, 3, 2, 0, 0, 2, 2, 0, 0, 0]

In [11]:
# 작성할 텍스트의 포트 설정
font = cv2.FONT_HERSHEY_PLAIN

# 인식된 사물에 대한 노이즈 제거작업(중복박스 제거)
indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
# boxes : 사각형 정보들
# confidences : 모든 박스들의 확률값
# min_confidence : 1차 임계값
# 0.4 : 동일 박스들에서 실제 사물에 해당되는 것을 제외한 나머지를 제거할 2차 임계값
#       (표시할 box의 내용이 담긴 인덱스만 추출)

for i in range(len(boxes)):
    for i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        print(i, label)
        color = colors[i]
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, label, (x, y+30), font, 2, (0, 255, 0), 1)
cv2.imshow("YOLO Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: only integer scalar arrays can be converted to a scalar index